# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [2]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import SimpleReader
import numpy as np
import h5py

import sys
import json
import requests
from IPython.display import IFrame

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start the server

In [11]:
%%bash --bg --proc bg_proc
python /Users/agurvich/research/repos/Firefly/FireflyFlaskApp.py

## Show Firefly in an IFrame

In [12]:
url = "http://localhost:5000"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [8]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("dummy.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to dummy.hdf5")

simple = SimpleReader("dummy.hdf5",write_jsons_to_disk=False)

Opening 1 files and 1 particle types...
JSONdir is None, defaulting to /Users/agurvich/research/repos/Firefly/static/data/Data
group1
posting...data posted!


## Send this data to the Flask app

*This sends the data.  (Then look back above to see it.)*

In [4]:
generic.sendDataViaFlask()

posting...data posted!


*To kill the Firefly server.*

In [9]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill


kill: 29931: No such process
